# Anomalie value definition

In [ ]:
!pip install -U kaleido

In [ ]:
import pandas as pd 
from tqdm import tqdm
import glob
import shutil
import numpy as np
import os
from PIL import Image
import pickle
from collections import Counter
from matplotlib import pyplot as plt
import plotly.graph_objects as go

import argparse
import cv2
import pandas as pd
from scipy.stats import ks_2samp
from tqdm.auto import tqdm


# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

def load_dataset(gt_paths_root, img_paths_root):

  img_tot_paths = []
  gt_tot_paths = []
  tot_labels = []
  tot_types = []

  for ix, img_path in enumerate(img_paths_root):
    defect_types = os.listdir(img_path)

    for defect_type in defect_types:
        if defect_type == 'good':
            img_paths = glob.glob(os.path.join(img_path, defect_type) + "/*.png")
            img_tot_paths.extend(img_paths)
            gt_tot_paths.extend([0]*len(img_paths))
            tot_labels.extend([0]*len(img_paths))
            tot_types.extend(['good']*len(img_paths))
        else:
            gt_path = gt_paths_root[ix]
            img_paths = glob.glob(os.path.join(img_path, defect_type) + "/*.png")
            gt_paths = glob.glob(os.path.join(gt_path, defect_type) + "/*.png")
            img_paths.sort()
            gt_paths.sort()
            img_tot_paths.extend(img_paths)
            gt_tot_paths.extend(gt_paths)
            tot_labels.extend([1]*len(img_paths))
            tot_types.extend([defect_type]*len(img_paths))

  assert len(img_tot_paths) == len(gt_tot_paths), "Something wrong with test and ground truth pair!"
  return img_tot_paths, gt_tot_paths, tot_labels, tot_types


## Old

In [ ]:
root = 'drive/MyDrive/data/mvtec_anomaly_detection'


categories = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 
              'hazelnut', 'leather', 'metalnut', 'pill', 'screw', 
              'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

cutting_window = 100
nearest_positive_neighbors = 10
save_path = 'drive/MyDrive/data/image_anomaly_value/results.pickle'

for category in categories:

  img_paths_root = [os.path.join(root, category, 'test')]
  gt_paths_root = [os.path.join(root, category,'ground_truth')]
  img_tot_paths, gt_tot_paths, tot_labels, tot_types = load_dataset(gt_paths_root, img_paths_root)

  with open(save_path, 'rb') as f:
    results = pickle.load(f)
  if category not in results:
    results[category] = {}
  print(category)

  for ix, p in tqdm(enumerate(img_tot_paths)):

    img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
    if tot_types[ix] == 'good' or img_name in results[category]:
      continue

    img = np.asarray(Image.open(p).convert('RGB'))
    img_gt = np.asarray(Image.open(gt_tot_paths[ix]).convert('RGB'))
    img_gt = img_gt[:,:,0]

    normal_indices = np.argwhere(img_gt == 0)
    anomaly_indices = np.argwhere(img_gt > 0)

    all_pixel_anomalies = [0] * normal_indices.shape[0]

    for a_i in anomaly_indices:
      mask = ((normal_indices[:, 0] >= max(a_i[0]-cutting_window, 0)) & 
              (normal_indices[:, 1] >= max(a_i[1]-cutting_window, 0)) & 
              (normal_indices[:, 0] <= min(a_i[0]+cutting_window, img.shape[0])) & 
              (normal_indices[:, 1] <= min(a_i[1]+cutting_window, img.shape[1]))
              )
      normal_indices_cut = normal_indices[mask]
      cutting_window_new = cutting_window + 20
      while len(normal_indices_cut) <= nearest_positive_neighbors:
        mask = ((normal_indices[:, 0] >= max(a_i[0]-cutting_window_new, 0)) & 
                (normal_indices[:, 1] >= max(a_i[1]-cutting_window_new, 0)) & 
                (normal_indices[:, 0] <= min(a_i[0]+cutting_window_new, img.shape[0])) & 
                (normal_indices[:, 1] <= min(a_i[1]+cutting_window_new, img.shape[1]))
                )
        normal_indices_cut = normal_indices[mask]
        cutting_window_new += 20

      a = np.square(normal_indices_cut - a_i).sum(axis=1)
      smallest_idx = np.argpartition(a,nearest_positive_neighbors)[:nearest_positive_neighbors]
      k_nearest_idx = normal_indices[smallest_idx, :]

      nearest_pixels = np.array([img[k[0],k[1],:] for k in k_nearest_idx]).astype(np.int16)
      pixel = img[a_i[0],a_i[1],:].astype(np.int16)

      pixel_anomaly = np.sqrt(((np.subtract(nearest_pixels,pixel)/255)**2).sum(axis=1)).mean()
      all_pixel_anomalies.append(pixel_anomaly)

    anomaly_score = np.mean(all_pixel_anomalies)

    results[category][img_name] = anomaly_score

    with open(save_path, 'wb') as f:
        pickle.dump(results, f)

# New values.
- Vb <-- Verhältnis zwischen Anormalen und Normalen Bilder
- Vk <-- Verhältnis zwischen Anormalen und Normalen Pixeln
- Ks <-- Kolgomorov-Smirnoff-Wert auf Basis von RGB-Histogrammen
- Vp2 <-- Verhältnis zwischen Anormalen Pixeln und Normalen Pixelnaus normalen bildern

In [ ]:
category = 'toothbrush'


In [ ]:
root = 'drive/MyDrive/data/mvtec_anomaly_detection'
img_paths_root = [os.path.join(root, category, 'test')]
gt_paths_root = [os.path.join(root, category,'ground_truth')]
img_tot_paths, gt_tot_paths, tot_labels, tot_types = load_dataset(gt_paths_root, img_paths_root)


In [ ]:
c = Counter(tot_labels)
Vb = c[0] / (c[0] + c[1])
Vb

In [ ]:
#for ix, p in tqdm(enumerate(img_tot_paths)):
Vk = 0
c = Counter(tot_labels)
cb = c[1]
good = []

for ix, p in tqdm(enumerate(img_tot_paths)):

  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')

  if tot_types[ix] == 'good':
    continue

  img = np.array(Image.open(p).convert('RGB'))
  img_gt = np.array(Image.open(gt_tot_paths[ix]).convert('RGB'))
  img_gt = img_gt[:,:,0]

  normal_indices = np.argwhere(img_gt == 0)
  anomaly_indices = np.argwhere(img_gt > 0)

  Vk += anomaly_indices.shape[0] / ((normal_indices.shape[0] + anomaly_indices.shape[0]))

print(Vk/cb)

In [ ]:
#for ix, p in tqdm(enumerate(img_tot_paths)):
KS = 0
KS2 = 0
c = Counter(tot_labels)
cb = c[1]
good = []
good_new = []

for ix, p in tqdm(enumerate(img_tot_paths)):

  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')

  if tot_types[ix] != 'good':
    img = np.array(Image.open(p).convert('RGB'))
    good.append(img)
    continue


for i, pic in enumerate(good):
  pic.setflags(write=1)
  is_na = np.expand_dims(np.logical_not(np.all(img<=35, axis=2)), axis=2)
  is_na_e = np.concatenate([is_na, is_na, is_na], axis=2)
  pic_new = np.where(is_na_e, pic, pic*np.nan)
  good_new.append(np.expand_dims(pic_new, axis=3))

gn = np.concatenate(good_new, axis=3)
gn_n = np.nan_to_num(np.nanmean(gn, axis=3))

for ix, p in enumerate(img_tot_paths):

  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
  if tot_types[ix] == 'good':
    continue

  img = np.asarray(Image.open(p).convert('RGB'))
  img_gt = np.asarray(Image.open(gt_tot_paths[ix]).convert('RGB'))
  img_gt = img_gt[:,:,0]

  normal_indices = np.argwhere(img_gt == 0)
  normal_indices2 = np.argwhere(gn_n > 0)
  anomaly_indices = np.argwhere(img_gt > 0)

  ks_i = 0
  ks_i2 = 0

  for channel in [0,1,2]:
    g = img[normal_indices[:, 0],normal_indices[:, 1], channel]
    g2 = img[normal_indices2[:, 0],normal_indices2[:, 1], channel]
    b = img[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
    ks_i += ks_2samp(g, b).statistic / 3
    ks_i2 += ks_2samp(g2, b).statistic / 3
  KS += ks_i
  KS2 += ks_i2
  
print(KS/cb)

print(KS2/cb)

In [ ]:
for ix, category in enumerate(categories):
  img_paths_root = [os.path.join(root, category, 'train')]
  img_tot_paths, gt_tot_paths, tot_labels, tot_types = load_dataset(gt_paths_root, img_paths_root)
  print(category, len(img_tot_paths))

# Putting it together

In [ ]:
save_path = 'drive/MyDrive/data/image_anomaly_value/detailed_scores.pickle'
root = 'drive/MyDrive/data/mvtec_anomaly_detection'
categories = sorted([i for i in os.listdir(root) if '.' not in i])

try:
  with open(save_path, 'rb') as f:
    results_overall = pickle.load(f)
except Exception as e:
  results_overall = {}

for ix, category in enumerate(categories):

  print(f'{ix+1}/{len(categories)}', category, end=' -- ')
  if category not in results_overall:
    results_overall[category] = []
  else:
    print(results_overall[category][-1]['KS5'])
    continue

  img_paths_root = [os.path.join(root, category, 'test')]
  gt_paths_root = [os.path.join(root, category,'ground_truth')]
  img_tot_paths, gt_tot_paths, tot_labels, tot_types = load_dataset(gt_paths_root, img_paths_root)

  c = Counter(tot_labels)

  # calculate the ratio of img normal to anormal
  Vb = c[0] / (c[0] + c[1])

  Vk = 0
  KS = 0
  KS2 = 0
  KS3 = 0
  KS4 = 0
  KS5 = 0
  a_type = {}
  gg = None
  good = []
  good_new = []

  for ix, p in enumerate(img_tot_paths):

    img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')

    if tot_types[ix] != 'good':
      img = np.array(Image.open(p).convert('RGB'))
      if gg is None:
        gg = img
      good.append(img)
      continue


  for i, pic in enumerate(good):
    pic.setflags(write=1)
    is_na = np.expand_dims(np.logical_not(np.all(img<=35, axis=2)), axis=2)
    is_na_e = np.concatenate([is_na, is_na, is_na], axis=2)
    pic_new = np.where(is_na_e, pic, pic*np.nan)
    good_new.append(np.expand_dims(pic_new, axis=3))

  gn = np.concatenate(good_new, axis=3)
  gn_n = np.nan_to_num(np.nanmean(gn, axis=3))[:,:,0]
  
  ggg = np.mean(np.concatenate([np.expand_dims(iii, axis=3) for iii in good], axis=3), axis=2)
  print(gg.shape)
  for ix, p in enumerate(img_tot_paths):

    img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
    l = tot_types[ix]
    if l == 'good':
      continue
    elif l not in a_type:
      a_type[l] = {'label': category,
                  'sub_category': l,
                  'num_anomalies': 0,
                  'Vb':c[0],
                  'Vk':0,
                  'KS':0,
                  'KS2':0,
                  'KS3':0,
                  'KS4':0,
                  'KS5':0}

    a_type[l]['num_anomalies'] += 1

    img = np.asarray(Image.open(p).convert('RGB'))
    img_gt = np.asarray(Image.open(gt_tot_paths[ix]).convert('RGB'))
    img_gt = img_gt[:,:,0]
    normal_indices = np.argwhere(img_gt == 0)
    normal_indices3 = np.argwhere(gn_n > 0)
    normal_indices2 = np.argwhere(np.logical_and(gn_n > 0, img_gt == 0))
    anomaly_indices = np.argwhere(img_gt > 0)

    # calculate the ratio of pixel normal to anormal
    vk_i = anomaly_indices.shape[0] / ((normal_indices.shape[0] + anomaly_indices.shape[0]))
    a_type[l]['Vk'] += vk_i
    Vk += vk_i

    # calculate actual rgb difference
    ks_i = 0
    ks_i2 = 0
    ks_i3 = 0
    ks_i4 = 0
    ks_i5 = 0
    for channel in [0,1,2]:
      g = img[normal_indices[:, 0],normal_indices[:, 1], channel]
      g2 = img[normal_indices2[:, 0],normal_indices2[:, 1], channel]
      g3 = img[normal_indices3[:, 0],normal_indices3[:, 1], channel]
      g4 = gg[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
      g5 = ggg[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
      b = img[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
      ks_i += ks_2samp(g, b).statistic / 3
      ks_i2 += ks_2samp(g2, b).statistic / 3
      ks_i3 += ks_2samp(g3, b).statistic / 3
      ks_i4 += ks_2samp(g4, b).statistic / 3
      ks_i5 += ks_2samp(g5, b).statistic / 3

    a_type[l]['KS'] += ks_i
    a_type[l]['KS2'] += ks_i2
    a_type[l]['KS3'] += ks_i3
    a_type[l]['KS4'] += ks_i4
    a_type[l]['KS5'] += ks_i5
    KS += ks_i
    KS2 += ks_i2
    KS3 += ks_i3
    KS4 += ks_i4
    KS5 += ks_i5

  for li in a_type:
    a_type[li]['Vb'] = a_type[li]['Vb'] / (a_type[li]['Vb'] + a_type[li]['num_anomalies'])
    a_type[li]['Vk'] = a_type[li]['Vk'] / a_type[li]['num_anomalies']
    a_type[li]['KS'] = a_type[li]['KS'] / a_type[li]['num_anomalies']
    a_type[li]['KS2'] = a_type[li]['KS2'] / a_type[li]['num_anomalies']
    a_type[li]['KS3'] = a_type[li]['KS3'] / a_type[li]['num_anomalies']
    a_type[li]['KS4'] = a_type[li]['KS4'] / a_type[li]['num_anomalies']
    a_type[li]['KS5'] = a_type[li]['KS5'] / a_type[li]['num_anomalies']
    results_overall[category].append(a_type[li])

  results_i = {'label': category,
               'sub_category': 'overall',
               'num_anomalies': c[1],
               'Vb': Vb,
               'Vk': Vk/c[1],
               'KS': KS/c[1],
               'KS2': KS2/c[1],
               'KS3': KS3/c[1],
               'KS4': KS4/c[1],
               'KS5': KS5/c[1]}

  results_overall[category].append(results_i)

  with open(save_path, 'wb') as f:
    pickle.dump(results_overall, f)

  print(results_overall[category][-1]['KS5'])

In [ ]:
import os
os.listdir()

In [ ]:
save_path = 'drive/MyDrive/data/image_anomaly_value/detailed_scores.pickle'
with open(save_path, 'rb') as f:
  results_overall = pickle.load(f)

d = []
for i in results_overall:
  d.extend(results_overall[i])

df = pd.DataFrame(d)

a = df[df['sub_category']=='overall']
#a.loc[:,'mean'] = a.loc[:, ['Vk', 'Vb', 'KS']].mean(axis=1)

a.rename(columns={
    'label': 'Kategorie',
    #'num_anomalies': 'Anz. Anomalien',
    'Vk': 'Vp'
    }, inplace=True
  )
a = a.drop(columns=['sub_category', 'num_anomalies']).set_index('Kategorie')
a

In [ ]:
import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.3) 
plt.figure(figsize=(22,2))
k = 'Vb'
g = sns.heatmap(a[k].sort_values(ascending=True).to_frame().T, annot=True, fmt='.3f', linewidths=.5, 
                cmap="Blues",
                annot_kws={"fontsize":18},
                cbar_kws = dict(use_gridspec=False, location="top", shrink=5.0, aspect=75))


g._axes.set_xticklabels(g._axes.get_xmajorticklabels(), fontsize = 20)
g._axes.set_yticklabels(g._axes.get_ymajorticklabels(), fontsize = 20)
g.collections[0].colorbar.ax.tick_params(labelsize=20)
plt.xticks(rotation=30)
#print(g.__dict__)
#g.figure.set_xticklabels(g.figure.get_xmajorticklabels(), fontsize = 16)

#plt.title('Anomaliewerte pro Kategorie des M
plt.savefig(f'{k.lower()}.png', dpi=300,bbox_inches='tight')

plt.show()

In [ ]:
import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

a = a[['Vb', 'Vp', 'KS']]
a['Ø'] = a.mean(axis=1)
a.sort_values('Ø', ascending=False, inplace=True)
df_plot = (a - a.min())/(a.max()-a.min())
sns.set(font_scale=1.3) 
plt.figure(figsize=(14,8))
g = sns.heatmap(df_plot, annot=a, cbar=False, fmt='.4f', linewidths=.5, 
                cmap="Blues",
                annot_kws={"fontsize":15})

g._axes.set_xticklabels(g._axes.get_xmajorticklabels(), fontsize = 16)
g._axes.set_yticklabels(g._axes.get_ymajorticklabels(), fontsize = 16)
#print(g.__dict__)
#g.figure.set_xticklabels(g.figure.get_xmajorticklabels(), fontsize = 16)

#plt.title('Anomaliewerte pro Kategorie des MVtec-AD Datensatz', loc='left', fontdict={'fontsize': 16, 'fontweight':'bold'})
plt.ylabel('Kategorie', fontsize=16)
plt.savefig('anomaliewerte.png', dpi=300)

plt.show()

In [ ]:
df = pd.DataFrame(d)
df['sub_category'].apply(lambda x: 'complete' if x == 'overall' else x)
df['label'] = df['label'] + ' - ' + df['sub_category']
df.rename(columns={
    'label': 'Kategorie',
    #'num_anomalies': 'Anz. Anomalien',
    'Vk': 'Vp'
    }, inplace=True
  )
df = df.drop(columns=['sub_category', 'num_anomalies']).set_index('Kategorie')
df

In [ ]:
import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

df_plot = (df - df.min())/(df.max()-df.min())
sns.set(font_scale=1.3) 
plt.figure(figsize=(14,28))
g = sns.heatmap(df_plot, annot=df, cbar=False, fmt='.4f', linewidths=.5, 
                cmap="Blues",
                annot_kws={"fontsize":13})
#print(g.__dict__)
#g.figure.set_xticklabels(g.figure.get_xmajorticklabels(), fontsize = 16)

#plt.title('Anomaliewerte pro Kategorie des MVtec-AD Datensatz', loc='left', fontdict={'fontsize': 16, 'fontweight':'bold'})
plt.ylabel('Kategorie', fontsize=16)
plt.tight_layout()
plt.savefig('anomaliewerte_complete.png', dpi=300)

plt.show()

# Plotting helper

In [ ]:
category = 'pill'
root = 'drive/MyDrive/data/mvtec_anomaly_detection'
img_paths_root = [os.path.join(root, category, 'test')]
gt_paths_root = [os.path.join(root, category,'ground_truth')]
img_tot_paths, gt_tot_paths, tot_labels, tot_types = load_dataset(gt_paths_root, img_paths_root)
print(gt_paths_root[0])
biggest_ks_i = []

for ix, p in tqdm(enumerate(img_tot_paths)):
  
  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
  l = tot_types[ix]
  if l == 'good':
    continue

  img = np.asarray(Image.open(p).convert('RGB'))
  img_gt = np.asarray(Image.open(gt_tot_paths[ix]).convert('RGB'))
  img_gt = img_gt[:,:,0]

  normal_indices = np.argwhere(img_gt == 0)
  anomaly_indices = np.argwhere(img_gt > 0)

  # calculate the ratio of pixel normal to anormal
  # vk_i = anomaly_indices.shape[0] / ((normal_indices.shape[0] + anomaly_indices.shape[0]))

    # calculate actual rgb difference
  ks_i = 0
  for channel in [0,1,2]:
    g = img[normal_indices[:, 0],normal_indices[:, 1], channel]
    b = img[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
    ks_i += ks_2samp(g, b).statistic / 3

  biggest_ks_i.append({'type':img_name.split('-')[0], 'number':img_name.split('-')[1], 'ks_i': ks_i})

df = pd.DataFrame(biggest_ks_i)
df.groupby('type').max().reset_index()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

img_pp = [
         [[
          "drive/MyDrive/data/mvtec_anomaly_detection/screw/test/thread_side/022.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/screw/ground_truth/thread_side/022_mask.png",
          "Schraube: thread_side - 0.9177"
         ],
         [
          "drive/MyDrive/data/mvtec_anomaly_detection/carpet/test/cut/016.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/carpet/ground_truth/cut/016_mask.png",
          "Teppich: cut - 0.1010"
         ]],
         [[
          "drive/MyDrive/data/mvtec_anomaly_detection/grid/test/bent/011.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/grid/ground_truth/bent/011_mask.png",
          "Schraube: thread_side - 0.2941"
         ],
         [
          "drive/MyDrive/data/mvtec_anomaly_detection/pill/test/pill_type/008.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/pill/ground_truth/pill_type/008_mask.png",
          "Teppich: cut - 0.9957"
         ]]

]

fig = make_subplots(rows=2, cols=6)
for iiii, img_p in enumerate(img_pp):
  for ix, i in enumerate(img_p):
    ####################### SCHRAUBE #######################
    i1 = np.array(Image.open(i[0]).convert('RGB'))
    g1 = np.asarray(Image.open(i[1]).convert('RGB'))
    img_gt = g1[:,:,0]

    normal_indices = np.argwhere(img_gt == 0)
    anomaly_indices = np.argwhere(img_gt > 0)
    fig.add_trace(go.Image(z=i1),
                  row=iiii+1, col=2*ix+1) # 0.8094
    fig.add_trace(go.Contour(z=img_gt, showscale=False,
                            contours=dict(start=0, end=700, size=70, coloring='lines'),
                            line_width=2), row=iiii+1, col=2*ix+1)
    for channel, color in enumerate(['rgba(187, 0, 0, 0.65)', 'rgba(10, 187, 39, 0.65)', 'rgba(26, 76, 175, 0.75)']):
      
      fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
      #fig.write_image("drive/MyDrive/data/mvtec_plots/hist_plots.png")

      g = i1[normal_indices[:, 0],normal_indices[:, 1], channel]
      histr = cv2.calcHist([g],[0],None,[256],[0,256])
      c = color
      if ix == 0:
        c = 'rgba(100, 100, 100, 0.75)'
        
      fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                              y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy', fillcolor=c,# 
                              legendgroup='green', mode='none', marker_color='green', line_color='green', showlegend=False 
                          ), row=iiii+1, col=2*ix+2+channel)
      #rgba(0, 33, 225, 0.65)

      b = i1[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
      histr = cv2.calcHist([b],[0],None,[256],[0,256])
      fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                              y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy', fillcolor='rgba(85, 20, 130, 0.75)',# 
                              legendgroup='red', mode='none', marker_color='red', line_color='red', showlegend=False 
                          ), row=iiii+1, col=2*ix+2+channel)
      if ix == 0:
        break

fig.update_layout(height=800, width=2050, title=i[2])
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.write_image("drive/MyDrive/data/mvtec_plots/hist_plots.png")
#fig.show()

capsule tests squeeze 014
pill test faulty_imprint 010
metalnut  test flip 018


# hist plot 2

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from PIL import Image

img_pp = [
          [
          "drive/MyDrive/data/mvtec_anomaly_detection/capsule/test/squeeze/014.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/capsule/ground_truth/squeeze/014_mask.png",
          "Kapsel - Zusammengedrückt <br>"
         ],
          [
          "drive/MyDrive/data/mvtec_anomaly_detection/pill/test/pill_type/008.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/pill/ground_truth/pill_type/008_mask.png",
          "Pille - Fehlerhafter Typ <br>"
         ],
          [
          "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/test/flip/018.png",
          "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/ground_truth/flip/018_mask.png",
          "Metall-Mutter - Umgedreht <br>"
         ]
        ]


fig = make_subplots(rows=2, cols=3, subplot_titles=("Plot 1", "Plot 2", "Plot 3"))
for iiii, i in enumerate(img_pp):
  
    ####################### SCHRAUBE #######################
  i1 = np.array(Image.open(i[0]).convert('RGB'))
  g1 = np.asarray(Image.open(i[1]).convert('RGB'))
  img_gt = g1[:,:,0]

  normal_indices = np.argwhere(img_gt == 0)
  anomaly_indices = np.argwhere(img_gt > 0)

  ks_i = 0

  for channel in [0,1,2]:
    g = i1[normal_indices[:, 0],normal_indices[:, 1], channel]
    b = i1[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
    ks_i += ks_2samp(g, b).statistic / 3

  fig.add_trace(go.Image(z=i1),
                row=1, col=iiii+1)
  fig.add_trace(go.Contour(z=img_gt, showscale=False,
                           contours=dict(start=0, end=700, size=70, coloring='lines'),
                           line_width=2), row=1, col=iiii+1)
  for channel, color in enumerate(['rgba(187, 0, 0, 0.65)', 'rgba(10, 187, 39, 0.65)', 'rgba(26, 76, 175, 0.75)']):
    
    g = i1[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]
    histr = cv2.calcHist([g.flatten()],[0],None,[256],[0,256])
      
    fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                            y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy', fillcolor=color,# 
                            mode='none', showlegend=False 
                        ), row=2, col=iiii+1)

  b = i1[normal_indices[:, 0],normal_indices[:, 1], channel]
  histr = cv2.calcHist([b.flatten()],[0],None,[256],[0,256])

  fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                          y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy', fillcolor='rgba(0, 0, 0, 0.8)',# 
                          mode='none',showlegend=False 
                      ), row=2, col=iiii+1)
  
  fig.layout.annotations[iiii].update(text=i[2] + f' KS:{round(ks_i,5)}')


fig.update_layout(height=800, width=1000)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.write_image("drive/MyDrive/data/mvtec_plots/hist2_plots.png")
#fig.show()



# Anomalie Values VK images

In [ ]:
from PIL import Image

grid = [
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/test/thread/010.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/ground_truth/thread/010_mask.png",
              "thread",
              0.0358
              ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/test/bent/011.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/ground_truth/bent/011_mask.png",
              "bent",
              0.0215
             ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/test/glue/010.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/grid/ground_truth/glue/010_mask.png",
              "glue",
              0.0073
             ]#,
             #[
             # "drive/MyDrive/data/mvtec_anomaly_detection/grid/test/metal_contamination/010.png",
             # "drive/MyDrive/data/mvtec_anomaly_detection/grid/ground_truth/metal_contamination/010_mask.png",
             # "metal_contamination",
             # 0.0081
             #]
        ]

transistor = [
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/test/bent_lead/009.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/ground_truth/bent_lead/009_mask.png",
              "bent_lead",
              0.0364
              ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/test/cut_lead/009.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/ground_truth/cut_lead/009_mask.png",
              "cut_lead",
              0.0358
             ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/test/damaged_case/009.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/transistor/ground_truth/damaged_case/009_mask.png",
              "damaged_case",
              0.0535
             ]#,
             #[
             # "drive/MyDrive/data/mvtec_anomaly_detection/transistor/test/misplaced/009.png",
             # "drive/MyDrive/data/mvtec_anomaly_detection/transistor/ground_truth/misplaced/009_mask.png",
             # "misplaced",
             # 0.4932
             #]
              ]

screw = [
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/test/manipulated_front/023.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/ground_truth/manipulated_front/023_mask.png",
              "manipulated_front",
              0.0066
              ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/test/scratch_head/023.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/ground_truth/scratch_head/023_mask.png",
              "scratch_head",
              0.0036
             ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/test/scratch_neck/024.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/screw/ground_truth/scratch_neck/024_mask.png",
              "scratch_neck",
              0.0071
             ]#,
             #[
             # "drive/MyDrive/data/mvtec_anomaly_detection/screw/test/thread_side/022.png",
             # "drive/MyDrive/data/mvtec_anomaly_detection/screw/ground_truth/thread_side/022_mask.png",
             # "scratch",
             # 0.0057
             #]
         ]

metal_nut = [
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/test/bent/024.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/ground_truth/bent/024_mask.png",
              "bent",
              0.0469
              ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/test/color/021.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/ground_truth/color/021_mask.png",
              "color",
              0.0656
             ],
             [
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/test/flip/022.png",
              "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/ground_truth/flip/022_mask.png",
              "flip",
              0.4872
             ]#,
             #[
             # "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/test/scratch/022.png",
             # "drive/MyDrive/data/mvtec_anomaly_detection/metalnut/ground_truth/scratch/022_mask.png",
             # "scratch",
             # 0.1230
             #]
             ]

fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20,11))
axs[0, 0].set_title('Metall-Mutter', fontsize=22)
for ix, i in enumerate(metal_nut):

  img = Image.open(i[0]).convert('RGB')
  img_gt = np.array(Image.open(i[1]).convert('RGB'))
  img_gt = img_gt[:,:,0]
  img_gt = np.ma.masked_where(img_gt < 0.9, img_gt)


  axs[ix, 0].imshow(img)
  axs[ix, 0].imshow(img_gt, interpolation='none', alpha=0.5, cmap='RdYlBu')
  axs[ix, 0].set_xlabel(f'{i[2]} - {i[3]}', fontsize=17)
  axs[ix, 0].axes.yaxis.set_ticklabels([])
  axs[ix, 0].axes.xaxis.set_ticklabels([])
  axs[ix, 0].grid(False)
  for tick in axs[ix, 0].xaxis.get_major_ticks():
    tick.tick1line.set_visible(False)
    tick.tick2line.set_visible(False)
    tick.label1.set_visible(False)
    tick.label2.set_visible(False)
  axs[ix, 0].axes.yaxis.set_visible(False)

axs[0, 1].set_title('Transistor', fontsize=22)
for ix, i in enumerate(transistor):

  img = Image.open(i[0]).convert('RGB')
  img_gt = np.array(Image.open(i[1]).convert('RGB'))
  img_gt = img_gt[:,:,0]
  img_gt = np.ma.masked_where(img_gt < 0.9, img_gt)


  axs[ix, 1].imshow(img)
  axs[ix, 1].imshow(img_gt, interpolation='none', alpha=0.5, cmap='RdYlBu')
  axs[ix, 1].set_xlabel(f'{i[2]} - {i[3]}', fontsize=17)
  axs[ix, 1].axes.yaxis.set_ticklabels([])
  axs[ix, 1].axes.xaxis.set_ticklabels([])
  axs[ix, 1].grid(False)
  for tick in axs[ix, 1].xaxis.get_major_ticks():
    tick.tick1line.set_visible(False)
    tick.tick2line.set_visible(False)
    tick.label1.set_visible(False)
    tick.label2.set_visible(False)
  axs[ix, 1].axes.yaxis.set_visible(False)

axs[0, 2].set_title('Schraube', fontsize=22)
for ix, i in enumerate(screw):

  img = Image.open(i[0]).convert('RGB')
  img_gt = np.array(Image.open(i[1]).convert('RGB'))
  img_gt = img_gt[:,:,0]
  img_gt = np.ma.masked_where(img_gt < 0.9, img_gt)


  axs[ix, 2].imshow(img)
  axs[ix, 2].imshow(img_gt, interpolation='none', alpha=0.5, cmap='RdYlBu')
  axs[ix, 2].set_xlabel(f'{i[2]} - {i[3]}', fontsize=20)
  axs[ix, 2].axes.yaxis.set_ticklabels([])
  axs[ix, 2].axes.xaxis.set_ticklabels([])
  axs[ix, 2].grid(False)
  for tick in axs[ix, 2].xaxis.get_major_ticks():
    tick.tick1line.set_visible(False)
    tick.tick2line.set_visible(False)
    tick.label1.set_visible(False)
    tick.label2.set_visible(False)
  axs[ix, 2].axes.yaxis.set_visible(False)


#axs[0, 3].set_title('Netz', fontsize=18)
#for ix, i in enumerate(grid):

#  img = Image.open(i[0]).convert('RGB')
#  img_gt = np.array(Image.open(i[1]).convert('RGB'))
#  img_gt = img_gt[:,:,0]
#  img_gt = np.ma.masked_where(img_gt < 0.9, img_gt)
#

#  axs[ix, 3].imshow(img)
#  axs[ix, 3].imshow(img_gt, interpolation='none', alpha=0.5, cmap='RdYlBu')
#  axs[ix, 3].set_xlabel(f'{i[2]} - {i[3]}', fontsize=14)
#  axs[ix, 3].axes.yaxis.set_ticklabels([])
#  axs[ix, 3].axes.xaxis.set_ticklabels([])
#  axs[ix, 3].grid(False)
#  for tick in axs[ix, 3].xaxis.get_major_ticks():
#    tick.tick1line.set_visible(False)
#    tick.tick2line.set_visible(False)
#    tick.label1.set_visible(False)
#    tick.label2.set_visible(False)
#  axs[ix, 3].axes.yaxis.set_visible(False)

plt.tight_layout()

plt.savefig("drive/MyDrive/data/mvtec_plots/vk_plots_2.png", dpi=100)

# Kovarianzanalyse der Werte mit Ergebnissen von patch_core und EfficientNets

In [ ]:
import plotly.express as px

_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metalnut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]

def extract_results(save_path_b7, save_path_b3, results_save_path_patch):
  results_b7 = {}
  results_b3 = {}
  for category in tqdm(_to_test):

    results_b7[category] = {}
    layer_combi_b7 = ["3", "4", "5"]
    results_save_path_b7 = f'{save_path_b7}/{category}/results_layer_{layer_combi_b7}.pickle'

    with open(results_save_path_b7, 'rb') as f:
      p = pickle.load(f)
      results_b7[category]["_".join(layer_combi_b7)] = p 


    results_b3[category] = {}
    layer_combi_b3 = ["5", "6", "7"]
    results_save_path_b3 = f'{save_path_b3}/{category}/results_layer_{layer_combi_b3}.pickle'

    with open(results_save_path_b3, 'rb') as f:
      p = pickle.load(f)
      results_b3[category]["_".join(layer_combi_b3)] = p 

    with open(results_save_path_patch, 'rb') as f:
      p = pickle.load(f)

    results_flattened = {'Kategorie': [],
                         'Original PatchCore': [],
                         'EfficientNet B7, 3_4_5, ABOD_n20, 10-mean': [],
                         'EfficientNet B3, 5_6_7, AutoEncoder, 512_264_64, 0.1': []
                         }
    for category, results_i in results_b7.items():
      results_flattened['Kategorie'].append(category)
      for layer_combi, algo_results in results_i.items():
        results_flattened['EfficientNet B7, 3_4_5, ABOD_n20, 10-mean'].append(algo_results['ABOD_20']['img_auc_top10'])
        
      results_flattened['EfficientNet B3, 5_6_7, AutoEncoder, 512_264_64, 0.1'].append(results_b3[category]['5_6_7']['512_264_64'][0.1][150])

      ccc = category if category != 'metalnut' else 'metal_nut'
      results_flattened['Original PatchCore'].append(p[ccc][20])


  return results_flattened


In [ ]:

save_path_b7 = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel_multi'
save_path_b3 = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_imglevel_multi'
results_save_path_patch = f'drive/MyDrive/data/wideresnet_evaluation/results_patchcore.pickle'
results_flattened = extract_results(save_path_b7, save_path_b3, results_save_path_patch)

df_scores = pd.DataFrame(results_flattened).set_index('Kategorie')
df_scores.sort_values(by='Original PatchCore', axis=0, ascending=False, inplace=True)

In [ ]:

save_path = 'drive/MyDrive/data/image_anomaly_value/detailed_scores.pickle'
with open(save_path, 'rb') as f:
  results_overall = pickle.load(f)

d = []
for i in results_overall:
  d.extend(results_overall[i])

df_an = pd.DataFrame(d)

df_overall = df_an[df_an['sub_category']=='overall']
#a.loc[:,'mean'] = a.loc[:, ['Vk', 'Vb', 'KS']].mean(axis=1)

df_overall.rename(columns={
    'label': 'Kategorie',
    #'num_anomalies': 'Anz. Anomalien',
    'Vk': 'Vp'
    }, inplace=True
  )
df_overall = df_overall.drop(columns=['sub_category', 'num_anomalies']).set_index('Kategorie')

In [ ]:
df_joined = df_scores.join(df_overall)
df_cov = df_joined.corr(method='kendall').loc[['Original PatchCore', 'EfficientNet B7, 3_4_5, ABOD_n20, 10-mean', 'EfficientNet B3, 5_6_7, AutoEncoder, 512_264_64, 0.1'], ['Vb', 'Vp', 'KS', 'KS2', 'KS3', 'KS4', 'KS5']]
df_cov.rename({'EfficientNet B7, 3_4_5, ABOD_n20, 10-mean': 'EfficientNet B7, 3_4_5,\nABOD_n20, 10-mean',
               'EfficientNet B3, 5_6_7, AutoEncoder, 512_264_64, 0.1': 'EfficientNet B3, 5_6_7, \nAutoEncoder, 512_264_64, 0.1'}, axis=0, inplace=True)
df_cov

In [ ]:

import numpy as np 
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.3) 
plt.figure(figsize=(14,5))
# annot=a,
g = sns.heatmap(df_cov, cbar=True, fmt='.4f', linewidths=.5, annot=True,
                cmap="PRGn", vmin=-1, vmax=1,
                annot_kws={"fontsize":16})

plt.yticks(fontsize= 18)

#plt.title('Anomaliewerte pro Kategorie des MVtec-AD Datensatz', loc='left', fontdict={'fontsize': 16, 'fontweight':'bold'})
plt.ylabel('Kategorie', fontsize=16)
plt.tight_layout()
plt.savefig('correlation.png', dpi=150)

plt.show()


# Vergleich anderer Ansätze

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
a = """PatchCore	99.6	98.4		95.0	true	[object Object]	2021-06-15
CFA	99.5	98.5			true	[object Object]	2022-06-09
Fastflow	99.4	98.5			true	[object Object]	2021-11-15
DRAEM+SSPCAB	98.9	97.2			false	[object Object]	2021-11-17
CS-Flow	98.7				true	[object Object]	2021-10-06
Reverse Distillation	98.5	97.8		93.9	true	[object Object]	2022-01-26
OCR-GAN	98.3				false	[object Object]	2022-03-01
CFLOW-AD	98.26	98.62		94.6	true	[object Object]	2021-07-27
MSPBA	98.1	98.1			true	[object Object]	2022-06-09
DRAEM	98.0	97.3			true	[object Object]	2021-08-17
PaDiM	97.9	97.5			true	[object Object]	2020-11-17
FYD	97.7	98.2			true	[object Object]	2021-10-09
PFM	97.5	97.3		93.0	true	[object Object]	2021-08-06
NSA	97.2	96.3			false	[object Object]	2021-09-30
CutPaste	97.1				true	[object Object]	2021-04-08
RSTPM	96.9	97.7			true	[object Object]	2021-11-30
CutPaste+SSPCAB	96.1				false	[object Object]	2021-11-17
AnoSeg	96	97			false	[object Object]	2021-10-07
Gaussian-AD	95.8				true	[object Object]	2020-05-28
STPM	95.5	97.0			true	[object Object]	2021-03-07
CutPaste	95.2	96.0			false	[object Object]	2021-04-08
RFS Energy	95.1				false	[object Object]	2021-08-27
InTra	95.0	96.6			false	[object Object]	2021-04-28
DifferNet	94.9				true	[object Object]	2020-08-28
f-AnoGAN	94.0				false	[object Object]	2017-03-17
DFR	93.8	95.5			true	[object Object]	2020-12-13
IGD	93.4	93.0			true	[object Object]	2021-01-25
GCPF	93.1	96.86			true	[object Object]	2021-01-06
Patch-SVDD	92.1	95.7			false	[object Object]	2020-06-29
RIAD	91.7	94.2			false	[object Object]	2020-10-17
UTAD	90				false	[object Object]	2021-03-22
MOCCA	87.5		0.88		false	[object Object]	2020-12-09
Mean-Shifted Contrastive Loss	87.2				true	[object Object]	2021-06-07
DisAug CLR	86.5	90.4			false	[object Object]	2020-11-04
RotNet (MLP Head)	86.3	93			false	[object Object]	2020-11-04
SPADE	85.5	96.5			true	[object Object]	2020-05-05
PEFM		98.30		95.52	true	[object Object]	2022-06-06
Semi-orthogonal		98.2			true	[object Object]	2021-05-31
IKD		97.81		92.55	true	[object Object]	2022-07-19
FAVAE		95.3			false	[object Object]	2020-08-12
FCDD (semi-supervised)		94			true	[object Object]	2020-07-03
CAVGA(weakly-supervised)		93			true	[object Object]	2019-11-19
VAE-grad		89.2			false	[object Object]	2020-02-10
CAVGA		89			false	[object Object]	2019-11-19
FCDD (unsupervised)		88			false	[object Object]	2020-07-03
AE-SSIM		87			false	[object Object]	2019-06-01
"""
#PatchCore	99.6	98.4		95.0	true	[object Object]	2021-06-15
data = {'Algorithmus_1': ['EN-B7, 3_4_5, ABOD_n20, 10mean',
                          'EN-B3, 5_6_6, AE_512_264_66'],
        'Algorithmus': ['EN-B7, 3_4_5, ABOD_n20, 10mean',
                          'EN-B3, 5_6_6, AE_512_264_66'],
        'AUC-ROC Wert': [0.9776, 0.9585],
        'Veröffentlichung': [pd.to_datetime("30.06.2022"), pd.to_datetime("30.06.2022")],
        ' ': ['Diese Arbeit', 'Diese Arbeit']}

for i, row in enumerate(a.split('\n')):
  row_s = row.split('\t')

  name = row_s[0]
  if name in ['f-AnoGAN', 'DisAug CLR']:
    continue
  auc = float(row_s[1])
  ver = pd.to_datetime(row_s[-1])
  
  data['Algorithmus_1'].append(name)
  if name in ['DRAEM+SSPCAB',
          'DREAM',
          'CFLOW-AD',
          'CutPaste+SSPCAB']:
    data['Algorithmus'].append('')
  else:
    data['Algorithmus'].append(name)
  #else:
  #  data['Algorithmus'].append('')
    
  data['AUC-ROC Wert'].append(auc/100)
  data['Veröffentlichung'].append(ver)
  data[' '].append('Externe Arbeiten')
  
  if name == 'SPADE':
    break



df = pd.DataFrame(data)
df.sort_values('AUC-ROC Wert', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates('Algorithmus_1', inplace=True)
#df = df.iloc[:-3,:]
df["compare"] = df.Veröffentlichung >= pd.to_datetime("03.11.2021", format='%d.%m.%Y')
df

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="Veröffentlichung", y="AUC-ROC Wert", text="Algorithmus", color=' ', color_discrete_sequence=px.colors.qualitative.Vivid[1:])
fig.update_traces(textposition='bottom center')
fig.update_layout(
    width=2400,
    height=1200,
    legend=dict(
        font_size=35,
        orientation="h",
        yanchor="bottom",
        y=1.0,
        xanchor="left",
        x=0
    ),
    font=dict(
        size=30
    )
)

def improve_text_position(x):
  """ it is more efficient if the x values are sorted """
  positions = ['Fastflow', 'Reverse Distillation', 'MSPBA', 'CFA', 'AnoSeg', 'CFLOW-AD']
  r = []
  for i in x:
    if i in positions:
      r.append('top center')
    else:
      r.append('bottom center')
  return r

fig.update_traces(textposition=improve_text_position(df['Algorithmus'])) 
fig.update_traces(textfont={'size': 30})
fig.update_traces(marker=dict(size=20))
fig.update_xaxes(title_font_size=40, range=[pd.to_datetime("14.04.2020"), pd.to_datetime("31.10.2022")], tickfont=dict(size=35))
fig.update_yaxes(title_font_size=40, tickfont=dict(size=35))

fig.show()
fig.write_image("drive/MyDrive/data/mvtec_plots/comparison.png", scale=1)

In [ ]:
from sklearn.metrics import roc_auc_score

a = np.array([0]*500 + [1] * 500)

b = np.array([0]*498 + [1, 1, 0, 0] + [1] * 498)

roc_auc_score(a,b)

# Some stuff

In [ ]:
for ix, p in enumerate(img_tot_paths):

  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
  image = cv2.imread(p)

In [ ]:

seen = 0

for ix, p in enumerate(img_tot_paths):

  img_name = p.split('test/')[1].replace('/', '-').replace('.png', '')
  image = cv2.imread(p)

  if tot_types[ix] == 'good' and seen:
    good_data = image
    continue
  
  if tot_types[ix] != 'good':
    # anomaly idx
    img = np.asarray(Image.open(p).convert('RGB'))
    img_gt = np.asarray(Image.open(gt_tot_paths[ix]).convert('RGB'))
    img_gt = img_gt[:,:,0]

    normal_indices = np.argwhere(img_gt == 0)
    anomaly_indices = np.argwhere(img_gt > 0)

  bad_data = image
  data = {'data': [],
          'Farbe': []}

  color = ('Blau', 'Rot', 'Grün')
  fig = go.Figure()
  for channel, col in enumerate(color):
      
      histr = cv2.calcHist([image],[channel],None,[256],[0,256])
      
      # data['data'].extend(histr.flatten().tolist())
      # data['Farbe'].extend([col for i in histr.flatten().tolist()])

      fig.add_trace(go.Scatter(x=[i for i in range(len(histr.flatten().tolist()))],
                              y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy',
                              name=col, mode='none' # override default markers+lines
                          ))
  fig.update_layout(
      autosize=False,
      width=1200,
      height=600,
      title={'text': "RGB-Histogram - "+ tot_types[ix]},
      legend=dict(orientation="h", x= 0.4, y= 1.1)
  )

  fig.update_xaxes(
          title_text = "Farbsättigung")

  fig.update_yaxes(
          title_text = "Anteil Pixel",
          title_standoff = 25)

  fig.show()
  if seen >= 1:
    break
  seen += 1

In [ ]:
from plotly.subplots import make_subplots

color = ('blue', 'red', 'green')
fig = make_subplots(rows=2, cols=2)

for channel, col in enumerate(color):
    
    histr = cv2.calcHist([good_data],[channel],None,[256],[0,256])

    fig.add_trace(go.Scatter(x=[i for i in range(len(histr.flatten().tolist()))],
                            y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy',
                            name=col, mode='none', marker=dict(color=col), line=dict(color=col), showlegend=True # override default markers+lines
                        ), row=1, col=1)
    
    histr = cv2.calcHist([bad_data],[channel],None,[256],[0,256])

    fig.add_trace(go.Scatter(x=[i for i in range(len(histr.flatten().tolist()))],
                            y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy',
                            name=col, mode='none', marker=dict(color=col), line=dict(color=col), showlegend=False 
                        ), row=1, col=2)
  
    g = bad_data[normal_indices[:, 0],normal_indices[:, 1], channel]
    b = bad_data[anomaly_indices[:, 0],anomaly_indices[:, 1], channel]

    histr = cv2.calcHist([g],[0],None,[256],[0,256])
    fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                            y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy',
                            name=col, mode='none', marker=dict(color=col), line=dict(color=col), showlegend=False 
                        ), row=2, col=1)
    
    histr = cv2.calcHist([b],[0],None,[256],[0,256])
    fig.add_trace(go.Scatter(x=[i for i in range(len(g))],
                            y=(histr.flatten()/sum(histr.flatten())).tolist(), fill='tozeroy',
                            name=col, mode='none', marker=dict(color=col), line=dict(color=col), showlegend=False 
                        ), row=2, col=2)


fig.update_layout(
    autosize=False,
    width=1200,
    height=1000,
    title={'text': "RGB-Histogram - "+ tot_types[ix]},
    legend=dict(orientation="h", x= 0.4, y= 1.1)
)

fig.update_xaxes(
        title_text = "Farbsättigung")

fig.update_yaxes(
        title_text = "Anteil Pixel",
        title_standoff = 25)

In [ ]:
g = bad_data[normal_indices[:, 0],normal_indices[:, 1], :]
b = bad_data[anomaly_indices[:, 0],anomaly_indices[:, 1], :]

In [ ]:
print((ks_2samp(g[:, 0],b[:, 0]).statistic + ks_2samp(g[:, 1],b[:, 1]).statistic + ks_2samp(g[:, 2],b[:, 2]).statistic)/3)
